In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# Load and preprocess data
pivot_df = pd.read_csv('fifty_temperatures_motes.csv')
pivot_df = pivot_df.apply(lambda x: x.fillna(x.mean()), axis=0)

num_nodes = 50
transmission_threshold = 0.5
penalty = 0.5
gamma = 1
pdr = 1.0
L = 1000
fixed_delta_t = 1
beta_1, beta_2 = 0.3, 0.3

# Define energy constants
E_max = 162000
E_t = 50 / 1000
E_s = 10 / 1000
E_w = 10 / 1000
E_0 = 1 / 1000

# DEWMA update function
def update_node_state_dewma(measured_value, last_state_value, last_rate_of_change, delta_t):
    x1 = beta_1 * measured_value + (1 - beta_1) * (last_state_value + last_rate_of_change * delta_t)
    x2 = beta_2 * (x1 - last_state_value) / delta_t + (1 - beta_2) * last_rate_of_change
    return x1, x2

# Function to calculate AoII
def calculate_aoii_sink(current_time, last_received_time, last_rate_of_change):
    return abs((current_time - last_received_time) * last_rate_of_change)

# Polling function with DEWMA
def poll_nodes_dewma(pivot_df, current_time, dewma_states_sink, actual_values, predicted_values, M, penalty, fixed_delta_t):
    whittle_arms = {}
    current_time_step = current_time

    # Calculate Whittle indices
    for mote in range(1, num_nodes + 1):
        column_name = f'mote{mote}'
        last_update_time = last_update_times[mote]
        dynamic_delta_t = current_time_step - last_update_time
        last_sink_state_value, last_sink_rate_of_change = dewma_states_sink[column_name]
        current_aoii = calculate_aoii_sink(current_time_step, last_received_times_sink[mote], last_sink_rate_of_change)
        future_aoii_passive = calculate_aoii_sink(current_time_step + 1, last_received_times_sink[mote], last_sink_rate_of_change)
        future_aoii_active = 0 if np.random.rand() < pdr else future_aoii_passive
        q_passive = current_aoii + gamma * future_aoii_passive
        q_active = current_aoii + gamma * future_aoii_active + penalty
        whittle_arms[mote] = q_passive - q_active

    # Select nodes based on Whittle index
    nodes_to_poll = [mote for mote in whittle_arms if whittle_arms[mote] >= 0]
    if len(nodes_to_poll) > M:
        nodes_to_poll = sorted(nodes_to_poll, key=whittle_arms.get, reverse=True)[:M]

    # Enforce L constraint
    nodes_exceeding_L = [mote for mote in range(1, num_nodes + 1) if (t - last_update_times[mote]) > L]
    if nodes_exceeding_L:
        if len(nodes_exceeding_L) <= M:
            nodes_to_poll = sorted(nodes_to_poll, key=whittle_arms.get, reverse=True)[:M - len(nodes_exceeding_L)]
            nodes_to_poll.extend(nodes_exceeding_L)
        else:
            nodes_to_poll = sorted(nodes_exceeding_L, key=lambda x: last_update_times[x], reverse=True)[:M]

    # Poll and update states
    for mote in nodes_to_poll:
        column_name = f'mote{mote}'
        measured_value = pivot_df.loc[current_time, column_name]
        delta_t = current_time_step - last_update_times[mote]
        actual_values[mote].append(measured_value)
        predicted_value_at_polling = dewma_states_sink[column_name][0] + dewma_states_sink[column_name][1] * delta_t
        predicted_values[mote].append(predicted_value_at_polling)
        last_state_value, last_rate_of_change = dewma_states_sink[column_name]
        dewma_states_sink[column_name] = update_node_state_dewma(measured_value, last_state_value, last_rate_of_change, fixed_delta_t)
        last_update_times[mote] = current_time
        last_received_times_sink[mote] = current_time
        polling_counter[mote] += 1

    return dewma_states_sink, actual_values, predicted_values

# Initialize settings
M_values = [1, 5, 10, 20]  # Multiple M values
results = {}

# Run simulation for each M value
for M in M_values:
    print(f"\nRunning simulation with M = {M}")
    dewma_states_sink = {f'mote{i}': np.array([20.0, 0.1]) for i in range(1, num_nodes + 1)}
    actual_values = {i: [] for i in range(1, num_nodes + 1)}
    predicted_values = {i: [] for i in range(1, num_nodes + 1)}
    polling_counter = {i: 0 for i in range(1, num_nodes + 1)}
    last_update_times = {i: 0 for i in range(1, num_nodes + 1)}
    last_received_times_sink = {i: 0 for i in range(1, num_nodes + 1)}

    num_steps = len(pivot_df)
    for t in range(num_steps):
        dewma_states_sink, actual_values, predicted_values = poll_nodes_dewma(
            pivot_df, t, dewma_states_sink, actual_values, predicted_values, M, penalty, fixed_delta_t
        )

    # Calculate polling fraction for each node
    total_polls = sum(polling_counter.values())
    polling_distribution = {node: count / total_polls for node, count in polling_counter.items()} if total_polls > 0 else {node: 0 for node in polling_counter}

    # Calculate MSE
    mse_per_node = {i: mean_squared_error(actual_values[i], predicted_values[i]) for i in range(1, num_nodes + 1)}
    total_mse = np.mean(list(mse_per_node.values()))
    
    results[M] = {
        "polling_distribution": polling_distribution,
        "total_mse": total_mse
    }
    print(f"M = {M}: Average MSE across all nodes: {total_mse:.4f}")

# Optional: plot polling distributions for each M
plt.figure(figsize=(10, 6))
for M in M_values:
    polling_distribution = results[M]["polling_distribution"]
    plt.plot(list(polling_distribution.keys()), list(polling_distribution.values()), label=f'M={M}')
plt.xlabel('Node ID')
plt.ylabel('Polling Fraction')
plt.legend()
plt.title('Polling Distribution for Different M Values')
plt.show()
